<a href="https://colab.research.google.com/github/btlgs2000/dl_intro/blob/master/keras_basics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Layer

In [62]:
from tensorflow.keras.layers import Dense, Layer, BatchNormalization, Dropout, concatenate
from tensorflow.keras import Sequential, Model, Input
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import Adam
import tensorflow.keras as keras

from sklearn.model_selection import train_test_split

import numpy as np

In [ ]:
dense = Dense(10)

In [ ]:
bn = BatchNormalization()

In [ ]:
isinstance(dense, Layer)

In [ ]:
# importante! settando questo attributo a False i suoi pesi non vengono aggiornati col gradient descent
dense.trainable

In [ ]:
dense.variables

In [ ]:
dense.name

In [ ]:
dense.input

In [ ]:
dense.output

In [ ]:
# inizializzazione dei pesi tramite build

In [ ]:
dense = Dense(10)

In [ ]:
dense.build((100,))

In [ ]:
dense.trainable_variables

In [ ]:
# settare l'inizializzazione

In [ ]:
dense = Dense(10, kernel_initializer='glorot_normal')

In [ ]:
dense = Dense(10, kernel_initializer=keras.initializers.GlorotNormal(seed=1))

In [ ]:
# inizializzazione mediante call

In [ ]:
dense = Dense(10)

In [ ]:
dense(np.random.rand(10, 45))

In [ ]:
bn(np.random.rand(10, 45))

In [ ]:
bn.weights

In [ ]:
bn.trainable_weights

In [ ]:
bn.non_trainable_weights

# Modelli

## Sequenziale

In [ ]:
# Posso specificare un input con il parametro input_shape del primo layer
# oppure includendo all'inizio un oggetto di tipo Input

model = Sequential(
    [
     Input(10),
     Dense(10, activation='relu', name='dense_1'),
     Dropout(0.1, name='dropout_1'),
     Dense(10, activation='relu', name='dense_2'),
     Dropout(0.1, name='dropout_2'),
     Dense(1, name='output')
    ]
)

In [ ]:
isinstance(model, Model)

In [ ]:
isinstance(model, Layer)

In [ ]:
model.layers

In [ ]:
dense_1 = model.get_layer(name='dense_1')

In [ ]:
dense_1.output

In [ ]:
dense_1.input

In [ ]:
model.summary()

In [ ]:
plot_model(model=model, show_shapes=True)

## Funzionale



In [ ]:
input_1 = Input(shape=(10,))
input_2 = Input(shape=(20,))

# prima sottorete
x = Dense(100, activation='relu')(input_1)
x = Dense(100, activation='relu')(x)
x = Dense(100, activation='relu')(x)
# seconda sottorete
y = Dense(100, activation='relu')(input_2)
y = Dense(100, activation='relu')(y)
y = Dense(100, activation='relu')(y)

conc = concatenate([x, y])
out = Dense(1)(conc)

model = Model(inputs=[input_1, input_2], outputs=out)

In [ ]:
model.summary()

In [ ]:
plot_model(model)

## Subclassing

In [45]:
class RegressorMLP(Model):
    def __init__(self, num_hidden_layers, activation, units_x_layer, dropout_rate):
        super().__init__(name='RegressorMLP')
        self.hidden_layers = []
        self.dropouts = []
        for i in range(num_hidden_layers):
            self.hidden_layers.append(Dense(units=units_x_layer, activation=activation, name=f'dense_{i}'))
            self.dropouts.append(Dropout(dropout_rate, name=f'dropout_{i}'))
        self.last_layer = Dense(1, name=f'dense_{num_hidden_layers}')

    def call(self, inputs, training=False):
        x = inputs
        for dense, dropout in zip(self.hidden_layers, self.dropouts):
            print(dense.name, dropout.name)
            x = dense(x)
            x = dropout(x) if training else x

        x = self.last_layer(x)
        
        return x

In [46]:
model = RegressorMLP(num_hidden_layers=4, activation='relu', units_x_layer=50, dropout_rate=0.3)

In [47]:
model(np.random.rand(32, 10).astype(np.float32), training=True)

dense_0 dropout_0
dense_1 dropout_1
dense_2 dropout_2
dense_3 dropout_3


<tf.Tensor: shape=(32, 1), dtype=float32, numpy=
array([[ 0.10910045],
       [ 0.45822078],
       [ 0.09547866],
       [ 0.05371245],
       [ 0.32558095],
       [ 0.10365595],
       [ 0.29161   ],
       [ 0.38242865],
       [ 0.06606817],
       [ 0.07444821],
       [ 0.18797219],
       [ 0.10027465],
       [-0.00575983],
       [ 0.20321989],
       [ 0.8152238 ],
       [ 0.24955243],
       [ 0.2227236 ],
       [-0.02099437],
       [ 0.14389856],
       [ 0.15358502],
       [ 0.38101864],
       [ 0.12439577],
       [ 0.04295134],
       [ 0.03579491],
       [ 0.19433002],
       [ 0.22707747],
       [ 0.27583084],
       [ 0.4445622 ],
       [ 0.12045928],
       [-0.13732302],
       [ 0.34698936],
       [ 0.13915986]], dtype=float32)>

# Addestramento

In [50]:
# come si addestra un modello

In [63]:
N = 100_000 # num of samples
x = np.random.rand(N, 10)
y = x @ np.array([1, 4, 2, 3, 5, 1, 6, 1, 1, 2]).reshape(-1, 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [64]:
model = Sequential(
    [
     Input(10),
     Dense(10, activation='relu', name='dense_1'),
     Dropout(0.1, name='dropout_1'),
     Dense(10, activation='relu', name='dense_2'),
     Dropout(0.1, name='dropout_2'),
     Dense(1, name='output')
    ]
)

In [65]:
model.compile(optimizer=Adam(learning_rate=1e-3), loss='mse', metrics=['mae'])

In [66]:
model.fit(x_train, y_train, batch_size=32, epochs=10, validation_split=0.2)

Epoch 1/10
2000/2000 [==============================] - 2s 1ms/step - loss: 15.6329 - mae: 2.6798 - val_loss: 0.2765 - val_mae: 0.4238
Epoch 2/10
2000/2000 [==============================] - 2s 1ms/step - loss: 4.5636 - mae: 1.6444 - val_loss: 0.3388 - val_mae: 0.4845
Epoch 3/10
2000/2000 [==============================] - 2s 1ms/step - loss: 3.5823 - mae: 1.4471 - val_loss: 0.1618 - val_mae: 0.3187
Epoch 4/10
2000/2000 [==============================] - 3s 1ms/step - loss: 2.7644 - mae: 1.2767 - val_loss: 0.1830 - val_mae: 0.3128
Epoch 5/10
2000/2000 [==============================] - 2s 1ms/step - loss: 2.2822 - mae: 1.1590 - val_loss: 0.1095 - val_mae: 0.2082
Epoch 6/10
2000/2000 [==============================] - 2s 1ms/step - loss: 1.8857 - mae: 1.0502 - val_loss: 0.1332 - val_mae: 0.2734
Epoch 7/10
2000/2000 [==============================] - 2s 1ms/step - loss: 1.5198 - mae: 0.9379 - val_loss: 0.0819 - val_mae: 0.2199
Epoch 8/10
2000/2000 [==============================] - 2s 1m

In [69]:
model.history.history

{'loss': [15.632884979248047,
  4.563602924346924,
  3.5823357105255127,
  2.76444149017334,
  2.2821505069732666,
  1.8856925964355469,
  1.5198496580123901,
  1.2320964336395264,
  1.018563151359558,
  0.8670998811721802],
 'mae': [2.6797659397125244,
  1.6444025039672852,
  1.4471186399459839,
  1.2766540050506592,
  1.159019947052002,
  1.0501712560653687,
  0.937853217124939,
  0.8390334248542786,
  0.7522034645080566,
  0.6898727416992188],
 'val_loss': [0.27648764848709106,
  0.33883994817733765,
  0.16176624596118927,
  0.18296580016613007,
  0.10946132242679596,
  0.1332128942012787,
  0.0819363072514534,
  0.06458050012588501,
  0.047976505011320114,
  0.04585960507392883],
 'val_mae': [0.42383456230163574,
  0.48448729515075684,
  0.318690687417984,
  0.31279829144477844,
  0.20819227397441864,
  0.2733917534351349,
  0.21992823481559753,
  0.19850261509418488,
  0.1655738353729248,
  0.13828973472118378]}

In [70]:
model.metrics_names

['loss', 'mae']

In [72]:
loss, mae = model.evaluate(x_test, y_test)
print(f'loss={loss}, mae={mae}')

625/625 [==============================] - 0s 745us/step - loss: 0.0467 - mae: 0.1373
loss=0.04671614244580269, mae=0.13726142048835754


In [75]:
y_hat = model.predict(x_test[:100])
print(y_hat.shape)

(100, 1)


# Salvataggio di un modello